In [34]:
#import os 
#os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
from EMAN2 import *
import numpy as np
import tensorflow as tf
import math

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

1 Physical GPUs, 1 Logical GPUs


In [2]:
# allocate 1000 test images as EMData objects
imgs=[test_image(size=(256,256)) for i in range(1000)]
for i in range(1000): imgs[i].add(test_image(1,size=(256,256)))

# convert the list to a single 1000x256x256 numpy array
imgsnp=np.stack([to_numpy(i) for i in imgs],axis=0)

# numpy -> tensorflow
imgstf=tf.constant(imgsnp)

In [27]:
%timeit fftstf=tf.signal.rfft2d(imgstf)

1.15 ms ± 82.4 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
%timeit ffts=[i.do_fft() for i in imgs]

181 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit fftsnp=np.fft.rfft2(imgsnp)

584 ms ± 6.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
cat=np.concatenate
# compute the 2-D FFT of each image
fftstf=tf.signal.rfft2d(imgstf)

# downsample via Fourier cropping
fftsclpy=tf.gather(fftstf,cat((np.arange(64),np.arange(192,256))),axis=1)
fftsclp=fftsclpy[:,:,:65]

In [4]:
# invert the FFT to get the downsampled real-space images
smaller=tf.signal.irfft2d(fftsclp)

1000

In [6]:
# back to EMData
imgsh=[from_tf(smaller[i]) for i in range(1000)]

In [7]:
EMData.write_images("xx.hdf:8",imgsh)
#for i in range(1000): imgsh[i].write_image("x.hdf:8",i)

RuntimeError: 

In [26]:
type(imgs[0])

libpyEMData2.EMData

In [58]:
ary=np.fromfunction(lambda x,y: np.int32(np.hypot(x,y)), (6,6))
ary

array([[0, 1, 2, 3, 4, 5],
       [1, 1, 2, 3, 4, 5],
       [2, 2, 2, 3, 4, 5],
       [3, 3, 3, 4, 5, 5],
       [4, 4, 4, 5, 5, 6],
       [5, 5, 5, 5, 6, 7]], dtype=int32)

In [ ]:
def fsc(f1,f2):
    

In [13]:
fftstf.ndim

3

In [20]:
t=Transform()
t.get_matrix()

[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

In [83]:
ny=16
ary=tf.constant(np.vstack((np.fromfunction(lambda y,x: np.int32(np.hypot(x,y)),(ny//2,ny//2+1)),np.fromfunction(lambda y,x: np.int32(np.hypot(x,ny//2-y)),(ny//2,ny//2+1)))))

In [84]:
ary

<tf.Tensor: shape=(16, 9), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 1,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 2,  2,  2,  3,  4,  5,  6,  7,  8],
       [ 3,  3,  3,  4,  5,  5,  6,  7,  8],
       [ 4,  4,  4,  5,  5,  6,  7,  8,  8],
       [ 5,  5,  5,  5,  6,  7,  7,  8,  9],
       [ 6,  6,  6,  6,  7,  7,  8,  9, 10],
       [ 7,  7,  7,  7,  8,  8,  9,  9, 10],
       [ 8,  8,  8,  8,  8,  9, 10, 10, 11],
       [ 7,  7,  7,  7,  8,  8,  9,  9, 10],
       [ 6,  6,  6,  6,  7,  7,  8,  9, 10],
       [ 5,  5,  5,  5,  6,  7,  7,  8,  9],
       [ 4,  4,  4,  5,  5,  6,  7,  8,  8],
       [ 3,  3,  3,  4,  5,  5,  6,  7,  8],
       [ 2,  2,  2,  3,  4,  5,  6,  7,  8],
       [ 1,  1,  2,  3,  4,  5,  6,  7,  8]], dtype=int32)>

In [85]:
fary=tf.signal.rfft2d(np.fromfunction(lambda y,x:x+y,(16,16)))

In [86]:
fary

<tf.Tensor: shape=(16, 9), dtype=complex64, numpy=
array([[3840.        +0.j      , -128.      +643.49945j ,
        -127.99999 +309.01932j , -127.999985+191.56552j ,
        -128.      +128.j      , -128.       +85.52687j ,
        -128.       +53.019325j, -128.       +25.460785j,
        -128.        +0.j      ],
       [-127.99999 +643.4995j  ,    0.        +0.j      ,
           0.        +0.j      ,    0.        +0.j      ,
           0.        +0.j      ,    0.        +0.j      ,
           0.        +0.j      ,    0.        +0.j      ,
           0.        +0.j      ],
       [-128.      +309.01935j ,    0.        +0.j      ,
           0.        +0.j      ,    0.        +0.j      ,
           0.        +0.j      ,    0.        +0.j      ,
           0.        +0.j      ,    0.        +0.j      ,
           0.        +0.j      ],
       [-128.      +191.56554j ,    0.        +0.j      ,
           0.        +0.j      ,    0.        +0.j      ,
           0.        +0.j      ,   